# ETL & Datawarehouse
**Purpose** : load, process and save data from a datalake (S3) to a datawarehouse (RDS)

In [15]:
# import 
import os
import boto3
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker 
from sqlalchemy import Column, Integer, String, Float 
from sqlalchemy.ext.declarative import declarative_base

In [16]:
# load dotenv variables
load_dotenv()
DBUSER = os.getenv('DBUSER')
DBPASS = os.getenv('DBPASS')
DBHOST = os.getenv('DBHOST')
DBNAME = os.getenv('DBNAME')

print(f"RDS Database : {DBNAME}")

RDS Database : postgres


In [17]:
# engine : connected to our db
connection_string = f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}"
engine = create_engine(connection_string, echo=True, future=True)


In [18]:
# sqlalchemy session to upload database
Session = sessionmaker(bind=engine)
local_session = Session()

In [19]:
# declarative base
Base = declarative_base()

# create city table
# Let's define our table 
class City(Base):
    __tablename__ = "cities"

    uuid         = Column(String, primary_key=True)
    name         = Column(String)
    full_address = Column(String)
    latitude     = Column(Float)
    longitude    = Column(Float)

    def __repr__(self):
        return f"<City(name={self.name})>"

# weather table
class Weather(Base):
    __tablename__ = "weathers"

    weather_id    = Column(Integer, primary_key=True)
    city_uuid     = Column(String)
    volume_rain   = Column(Float)

    def __repr__(self):
        return f"<Weather(uuid={self.city_uuid}, volume={self.volume_rain})>"

# hotel table
class Hotel(Base):
    __tablename__ = "hotels"

    hotel_id     = Column(Integer, primary_key=True)
    name        = Column(String)
    city        = Column(String)
    url         = Column(String)
    image_url   = Column(String)
    score       = Column(Float)
    description = Column(String)

    def __repr__(self):
        return f"<Hotel(name={self.name}, score={self.score})>"

In [20]:
# create_table
INIT = False
if INIT : 
    Base.metadata.create_all(engine)

### Read Datalake files
* weather_csv
* hotels_booking_csv

In [21]:
# s3 bucket
# dotenv variables
load_dotenv()
AWSS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

# aws session
session = boto3.Session(aws_access_key_id=AWSS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# s3 service
s3 = session.resource("s3")

# bucket 
bucket_name = 'kayak-mahadou'
kayak_bucket = s3.Bucket(bucket_name)

In [22]:
# read weather data from s3
weather_key = "weather_data.csv"
weather_csv_obj = s3.Object(bucket_name, weather_key)
weather_csv = weather_csv_obj.get()['Body'].read().decode('utf-8') 

# dataframe
weather_df = pd.read_csv(weather_csv, index_col=[0])
display(weather_df.sample(2))
weather_df.columns

,uuid,cities,full_address,latitude,longitude,volume_rain_7days
32,176ebf7f-eef4-44bd-82d3-4754e153833e,Biarritz,"Biarritz, Bayonne, Pyrénées-Atlantiques, Nouve...",43.471144,-1.552727,9.181
21,1af774b6-52ae-4c60-b371-bc10d6a33c44,Aix en Provence,"Aix-en-Provence, Bouches-du-Rhône, Provence-Al...",43.529842,5.447474,1.273


Index(['uuid', 'cities', 'full_address', 'latitude', 'longitude',
       'volume_rain_7days'],
      dtype='object')

In [23]:
# read hotels data from s3
hotels_key = "hotels_booking.csv"
hotels_key_obj = s3.Object(bucket_name, hotels_key)
hotels_csv = hotels_key_obj.get()['Body'].read().decode('utf-8') 

# dataframe
hotels_df = pd.read_csv(hotels_csv, index_col=[0])
display(hotels_df.sample(2))

,city,name,url,image_url,score,description
12,le-havre,Holiday Inn Express - Le Havre Centre,booking.com/hotel/fr/campanile-le-havre-centre...,https://cf.bstatic.com/xdata/images/hotel/squa...,8.2,"L’Holiday Inn Express - Le Havre Centre, situé..."
135,colmar,Hôtel Roi Soleil Colmar,booking.com/hotel/fr/roi-soleil.fr.html?sid=08...,https://cf.bstatic.com/xdata/images/hotel/squa...,7.8,L'Hôtel Roi Soleil Colmar est installé dans le...


## ETL

In [24]:
# city df
keep_col = ['uuid', 'cities', 'full_address', 'latitude', 'longitude']
city_df = weather_df[keep_col].copy()

In [25]:
# weather df
keep_col = ['uuid', 'volume_rain_7days']
weather_df_volume = weather_df[keep_col].copy()
weather_df_volume = weather_df_volume.reset_index()
display(weather_df_volume.sample(2))

,index,uuid,volume_rain_7days
3,3,ca21cdf4-e86e-4d3c-a894-f75e6ce20fe6,5.985
26,26,25831779-40dd-4f9f-95ea-cb9cb26a506c,1.227


In [26]:
# hotel dataframe
hotels_df = hotels_df.reset_index().copy()
display(hotels_df.sample(2))

,index,city,name,url,image_url,score,description
289,289,la-rochelle,Mercure La Rochelle Vieux Port Sud,booking.com/hotel/fr/la-rochelle-vieux-port-su...,https://cf.bstatic.com/xdata/images/hotel/squa...,7.9,Doté d'un restaurant panoramique avec vue sur ...
156,156,cassis,Le Clos Des Arômes,booking.com/hotel/fr/le-clos-des-aromes.fr.htm...,https://cf.bstatic.com/xdata/images/hotel/squa...,8.4,"Situé à Cassis, l'établissement Le Clos Des Ar..."


## DataWarehouse
Tables : 
* cities
* weathers
* hotels

### City

In [27]:
# # all_cities in database
# cities_list = local_session.query(City)..filter_by(name='Joe').first()
# session.query(db.users).filter_by(name='Joe', surname='Dodson')
# user = User.query.filter_by(email=email).first()

In [28]:
# fill City TABLE
for idx in range(city_df.shape[0]) : 
    # get row
    city_row = city_df.iloc[idx]

    # create city object
    city_uuid         = city_row["uuid"]
    city_name         = city_row["cities"]
    city_fulladdress  = city_row["full_address"]
    city_latitude     = city_row["latitude"]
    city_longitude    = city_row["longitude"]
    city = City(uuid=city_uuid, name=city_name, full_address=city_fulladdress,
                latitude=city_latitude, longitude=city_longitude)

    # add & commit
    # verify if cities in table
    city_in_table = local_session.query(City).filter_by(name=city_name).first()

    if not city_in_table : 
        # Add values to db 
        local_session.add(city)

        # Commit the results 
        local_session.commit()   
        
    print(city)

2021-12-04 23:43:45,691 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2021-12-04 23:43:45,693 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-12-04 23:43:45,701 INFO sqlalchemy.engine.Engine select current_schema()
2021-12-04 23:43:45,702 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-12-04 23:43:45,711 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-12-04 23:43:45,713 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-12-04 23:43:45,721 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-04 23:43:45,732 INFO sqlalchemy.engine.Engine SELECT cities.uuid AS cities_uuid, cities.name AS cities_name, cities.full_address AS cities_full_address, cities.latitude AS cities_latitude, cities.longitude AS cities_longitude 
FROM cities 
WHERE cities.name = %(name_1)s 
 LIMIT %(param_1)s
2021-12-04 23:43:45,734 INFO sqlalchemy.engine.Engine [generated in 0.00180s] {'name_1': 'Mont Saint Michel', 'param_1': 1}
<City(name=Mont Saint Michel)>
2021-12-04 23:43:45,744 INF

### Weather

In [29]:
# fill Weather TABLE
for idx in range(weather_df_volume.shape[0]) : 
    # get row
    weather_row = weather_df_volume.iloc[idx]

    # create city object
    weather_id        = int(weather_row["index"])
    city_uuid         = weather_row["uuid"]
    weather_volume    = float(weather_row["volume_rain_7days"])
    weather = Weather(weather_id=weather_id, city_uuid=city_uuid, volume_rain=weather_volume)

    # add & commit
    # verify if cities in table
    weather_in_table = local_session.query(Weather).filter_by(weather_id=weather_id).first()

    if not weather_in_table : 
        # Add values to db 
        local_session.add(weather)

        # Commit the results 
        local_session.commit()   
        
    print(weather)

2021-12-04 23:43:48,635 INFO sqlalchemy.engine.Engine SELECT weathers.weather_id AS weathers_weather_id, weathers.city_uuid AS weathers_city_uuid, weathers.volume_rain AS weathers_volume_rain 
FROM weathers 
WHERE weathers.weather_id = %(weather_id_1)s 
 LIMIT %(param_1)s
2021-12-04 23:43:48,637 INFO sqlalchemy.engine.Engine [generated in 0.00264s] {'weather_id_1': 0, 'param_1': 1}
<Weather(uuid=87f71cf6-82e0-4d02-b54e-e291624d4a7a, volume=7.118)>
2021-12-04 23:43:48,648 INFO sqlalchemy.engine.Engine SELECT weathers.weather_id AS weathers_weather_id, weathers.city_uuid AS weathers_city_uuid, weathers.volume_rain AS weathers_volume_rain 
FROM weathers 
WHERE weathers.weather_id = %(weather_id_1)s 
 LIMIT %(param_1)s
2021-12-04 23:43:48,649 INFO sqlalchemy.engine.Engine [cached since 0.01448s ago] {'weather_id_1': 1, 'param_1': 1}
<Weather(uuid=ca7c0a3f-29b6-40d0-b546-4db8b5b6a903, volume=6.299)>
2021-12-04 23:43:48,656 INFO sqlalchemy.engine.Engine SELECT weathers.weather_id AS weathers

### Hotel

In [30]:
# fill Hotel TABLE
for idx in range(hotels_df.shape[0]) : 
    # get row
    hotel_row = hotels_df.iloc[idx]

    # create city object
    hotel_id        = int(hotel_row["index"])
    name            = hotel_row["name"]
    city            = hotel_row["city"]
    url             = hotel_row["url"]
    image_url       = hotel_row["image_url"]
    score           = hotel_row["score"]
    description     = hotel_row["description"]

    # hotel object
    hotel = Hotel(hotel_id=hotel_id, name=name, 
                    city=city, url=url, 
                    image_url=image_url, score=score, 
                    description=description)

    # add & commit
    # verify if cities in table
    hotel_in_table = local_session.query(Hotel).filter_by(name=name).first()

    if not hotel_in_table : 
        # Add values to db 
        local_session.add(hotel)

        # Commit the results 
        local_session.commit()   
        
    print(hotel)

2021-12-04 23:43:49,605 INFO sqlalchemy.engine.Engine SELECT hotels.hotel_id AS hotels_hotel_id, hotels.name AS hotels_name, hotels.city AS hotels_city, hotels.url AS hotels_url, hotels.image_url AS hotels_image_url, hotels.score AS hotels_score, hotels.description AS hotels_description 
FROM hotels 
WHERE hotels.name = %(name_1)s 
 LIMIT %(param_1)s
2021-12-04 23:43:49,608 INFO sqlalchemy.engine.Engine [generated in 0.00317s] {'name_1': 'Les Terrasses Poulard', 'param_1': 1}
<Hotel(name=Les Terrasses Poulard, score=7.3)>
2021-12-04 23:43:49,621 INFO sqlalchemy.engine.Engine SELECT hotels.hotel_id AS hotels_hotel_id, hotels.name AS hotels_name, hotels.city AS hotels_city, hotels.url AS hotels_url, hotels.image_url AS hotels_image_url, hotels.score AS hotels_score, hotels.description AS hotels_description 
FROM hotels 
WHERE hotels.name = %(name_1)s 
 LIMIT %(param_1)s
2021-12-04 23:43:49,624 INFO sqlalchemy.engine.Engine [cached since 0.0198s ago] {'name_1': 'Le Relais Saint Michel', '